In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive/RAG-Chatbot-Project/

/content/drive/My Drive/RAG-Chatbot-Project


In [3]:
!pip uninstall -y langchain langchain-core langchain-community langchain-google-genai langchain-text-splitters google-generativeai
!pip install -U --force-reinstall langchain langchain-core langchain-community langchain-google-genai langchain-text-splitters google-generativeai
!pip install -q sentence-transformers chromadb pypdf

Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.79
Uninstalling langchain-core-0.3.79:
  Successfully uninstalled langchain-core-0.3.79
Found existing installation: langchain-text-splitters 0.3.11
Uninstalling langchain-text-splitters-0.3.11:
  Successfully uninstalled langchain-text-splitters-0.3.11
Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible w

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [4]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

DATA_PATH = './data'

loader = DirectoryLoader(DATA_PATH, glob="*.txt", loader_cls=TextLoader)

documents = loader.load()

print(f"Loaded {len(documents)} document(s).")
print(f"First document content: {documents[0].page_content[:100]}...") # Print first 100 chars

Loaded 2 document(s).
First document content: "Hello World"...


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
print(f"Example chunk: {chunks[0].page_content}")

Split 2 documents into 2 chunks.
Example chunk: "Hello World"


In [6]:
!pip install -q chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.6 MB/s e

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

persist_directory = './chroma_db'

vectorstore = Chroma.from_documents(
    chunks,
    embedding_model,
    persist_directory=persist_directory
)

print(f"Successfully created and saved vector store to '{persist_directory}'.")

Successfully created and saved vector store to './chroma_db'.


In [8]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

# Get your API key from secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')

# Set up the Gemini-Pro model
llm = ChatGoogleGenerativeAI(model="gemini-pro")

print("Gemini LLM is loaded and ready.")

ModuleNotFoundError: No module named 'langchain_core.pydantic_v1'

In [14]:
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer concise.

Context: {context}
Question: {question}
Helpful Answer:
"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True
)

print("Successfully created the RAG chain.")

ModuleNotFoundError: No module named 'langchain.chains'

In [ ]:
question = "What is the message?"
result = qa_chain.invoke({"query": question})

print(f"Question: {result['query']}\n")
print(f"Answer: {result['result']}\n")
print(f"Source Document: {result['source_documents'][0].page_content}")

In [ ]:
question = "What is the James Webb Space Telescope?"
result = qa_chain.invoke({"query": question})

print(f"Question: {result['query']}\n")
print(f"Answer: {result['result']}\n")
print(f"Source Document: {result['source_documents'][0].page_content}")

In [9]:
!git config --global user.email "karthikk1162@gmail.com"
!git config --global user.name "Karthik K"